In [1]:
from load_data_json import LoadDataset 
from preprocess import VectorSpaceModel
from preprocess_restart import VectorSpaceModel2
import numpy as np
import time
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds, eigsh
import ipyparallel as ipp
from scipy.sparse import diags
import math


## Load the dataset
Functions to load the dataset and the given queries with associated relevant documents.

In [2]:
dataset = LoadDataset("./data/npl/npl.json", "./data/npl/queries.json", "./data/npl/qrels-treceval.txt","npl")
# vsm = VectorSpaceModel(dataset.doc_matrix, dataset.name)
vsm = VectorSpaceModel2(dataset.doc_matrix, dataset.name)

c:\Users\dungl\anaconda3\envs\IR_VectorSpaceModel\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [3]:
vsm.parallel_lanczos(dataset.query_vectors)

Starting 2 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/2 [00:00<?, ?engine/s]

<lambda>:   0%|          | 0/2 [00:00<?, ?tasks/s]

respone_wrapper:   0%|          | 0/2 [00:00<?, ?tasks/s]

respone_wrapper:   0%|          | 0/2 [00:00<?, ?tasks/s]

...

respone_wrapper:   0%|          | 0/2 [00:00<?, ?tasks/s]

respone_wrapper:   0%|          | 0/2 [00:00<?, ?tasks/s]

respone_wrapper:   0%|          | 0/2 [00:00<?, ?tasks/s]

respone_wrapper:   0%|          | 0/2 [00:00<?, ?tasks/s]

Stopping engine(s): 1714887078
engine set stopped 1714887078: {'engines': {'0': {'exit_code': 1, 'pid': 9660, 'identifier': '0'}, '1': {'exit_code': 1, 'pid': 4476, 'identifier': '1'}}, 'exit_code': 1}
Stopping controller
Controller stopped: {'exit_code': 1, 'pid': 13260, 'identifier': 'ipcontroller-1714887076-7hx9-14500'}


In [4]:
vsm.parallel_lanczos(dataset.query_vectors, dc=4)

Starting 4 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/4 [00:00<?, ?engine/s]

<lambda>:   0%|          | 0/4 [00:00<?, ?tasks/s]

respone_wrapper:   0%|          | 0/4 [00:00<?, ?tasks/s]

respone_wrapper:   0%|          | 0/4 [00:00<?, ?tasks/s]

...

respone_wrapper:   0%|          | 0/4 [00:00<?, ?tasks/s]

respone_wrapper:   0%|          | 0/4 [00:00<?, ?tasks/s]

respone_wrapper:   0%|          | 0/4 [00:00<?, ?tasks/s]

respone_wrapper:   0%|          | 0/4 [00:00<?, ?tasks/s]

Stopping engine(s): 1714887736
engine set stopped 1714887736: {'engines': {'0': {'exit_code': 1, 'pid': 5612, 'identifier': '0'}, '1': {'exit_code': 1, 'pid': 7764, 'identifier': '1'}, '2': {'exit_code': 1, 'pid': 16160, 'identifier': '2'}, '3': {'exit_code': 1, 'pid': 1324, 'identifier': '3'}}, 'exit_code': 1}
Stopping controller
Controller stopped: {'exit_code': 1, 'pid': 9016, 'identifier': 'ipcontroller-1714887735-wb7y-14500'}
